Basado sobre https://medium.com/mlearning-ai/transfer-learning-with-transformers-trainer-and-pipeline-for-nlp-8b1d2c1a8c3d

In [2]:
!pip install datasets

     |████████████████████████████████| 441 kB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 163 kB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 212 kB 17.0 MB/s eta 0:00:01
     |████████████████████████████████| 35.3 MB 9.7 MB/s eta 0:00:011     |████████████████▊               | 18.4 MB 10.0 MB/s eta 0:00:02
     |████████████████████████████████| 132 kB 12.3 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 260 kB/s  eta 0:00:01
ERROR: multiprocess 0.70.14 has requirement dill>=0.3.6, but you'll have dill 0.3.5.1 which is incompatible.
ERROR: responses 0.18.0 has requirement urllib3>=1.25.10, but you'll have urllib3 1.25.8 which is incompatible.
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.1.1
    Uninstalling huggingface-hub-0.1.1:
      Successfully uninstalled huggingface-hub-0.1.1


In [1]:
from datasets import load_dataset

raw_datasets = load_dataset("imdb")

Found cached dataset imdb (/home/mvernier/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
raw_datasets["train"][58]

{'text': 'I thought this was a very clunky, uninvolving version of a famous Australian story. Heath Ledger and Orlando Bloom were very good in their roles, and gave their characters some personality; but the whole thing felt forced and mechanical.<br /><br />The beginning could have been a lot more involving; perhaps starting with a shootout, and then flashing back for a recap of how they got there or that sort of thing. And I felt like every scene was routinely predictable and signposted, like a very bad tv soap.<br /><br />I was really looking forward to this movie, and hoping for something a lot better. The only thing I can say in its favour is that it beats the Mick Jagger version, but not by much.',
 'label': 0}

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

2022-11-08 12:17:09.915130: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-08 12:17:10.446363: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-08 12:17:10.446384: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-08 12:17:10.483327: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-08 12:17:18.191761: W tensorflow/stream_executor/platform/de

  0%|          | 0/25 [00:00<?, ?ba/s]

Loading cached processed dataset at /home/mvernier/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-fa6d3f558cc9c714.arrow


In [10]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))

small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(10))

Loading cached shuffled indices for dataset at /home/mvernier/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-8c06c8997e0fe74e.arrow
Loading cached shuffled indices for dataset at /home/mvernier/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-4f32996f7be42fb4.arrow


In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

from transformers import Trainer

trainer = Trainer(model=model, args=training_args, train_dataset=small_train_dataset, eval_dataset=small_eval_dataset)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [12]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 10
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6


RuntimeError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 3.95 GiB total capacity; 2.94 GiB already allocated; 7.81 MiB free; 2.99 GiB reserved in total by PyTorch)

In [8]:
trainer.save_model()

In [13]:
from transformers import pipeline# load from previously saved model
pipe = pipeline("text-classification", model="test_trainer", tokenizer="bert-base-cased")
pipe("This restaurant is awesome")

file test_trainer/config.json not found


OSError: Can't load config for 'test_trainer'. Make sure that:

- 'test_trainer' is a correct model identifier listed on 'https://huggingface.co/models'
  (make sure 'test_trainer' is not a path to a local directory with something else, in that case)

- or 'test_trainer' is the correct path to a directory containing a config.json file

